In [2]:
from fundAccess import searchEdgarFund,searchEdgarNCSR
from secReportSweeping import filingSplitPages, searchPagesForPatterns, savePagesTuples, associateFilingPagesToElements

In [ ]:
import numpy as np
!pip install pyhtml2pdf
!pip install pdfkit
!pip install PyMuPDF # this will allow you to use fitz
import fitz
import pandas as pd
#!pip install wkhtmltopdf
!sudo apt-get install wkhtmltopdf
!pip install PyPDF2
!pip install rank-bm25
!apt-get update
#!wget https://github.com/wkhtmltopdf/packaging/releases/download/0.12.6.1-2/wkhtmltox_0.12.6.1-2.jammy_amd64.deb
#!apt install /content/wkhtmltox_0.12.6.1-2.jammy_amd64.deb && rm /content/wkhtmltox_0.12.6.1-2.jammy_amd64.deb
%pip install anytree
from anytree import Node, RenderTree
from rank_bm25 import BM25Okapi
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import string
!pip install rouge

nltk.download('punkt')
nltk.download('stopwords')

## Main Functions
run it

In [3]:
import requests
from bs4 import BeautifulSoup
from rank_bm25 import BM25Okapi
from itertools import islice
import PyPDF2
import pdfkit
import re

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import PorterStemmer

# Tokenization
def tokenize(text):
    words = nltk.word_tokenize(text)
    return [word for word in words if word.isalnum()]

# Remove stopwords
stop_words = set(stopwords.words("english"))
def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])


# Lemmatization
lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])


# Stemming
stemmer = PorterStemmer()
def stem_text(text):
    return ' '.join([stemmer.stem(word) for word in text.split()])

# Preprocessing pipeline
def preprocess(text):
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation and special characters
    text = ''.join(char for char in text if char.isalnum() or char.isspace())

    # Remove stopwords
    text = remove_stopwords(text)

    return text


def cleaning(aText):

    #remove not some not alphanumeric character
    t  = re.sub("[^a-zA-Z0-9_\s\-%.&\,:;@\(\)$\x92\'\ʼ\ʻ\’\\\/\*]", ' ', aText)

    #substitute line break by space
    t  = re.sub("\n+", " ", t )

    #combine multiple space into 1 space
    t = re.sub("\s+", " ", t )

    t  = re.sub("\xa0+", " ", t )

    t = t.strip()

    #'
    #t  = re.sub("\x92", "'", t )

    #remove ascii
    #t = re.sub(r'[^\x00-\x7F]', ' ', t)

    return t

def get_report(url,name='result'):
  # URL of the HTML report
  url = url

  headers = {"User-Agent": "Mozilla/5.0"}
  response = requests.get(url,headers=headers)

  # Check if the request was successful
  if response.status_code == 200:
      # Parse the HTML content using BeautifulSoup
      soup = BeautifulSoup(response.text, 'html.parser')


      with open(name+'.html','w') as f:
        f.write(response.text)


      with open(name+'.html','r') as f:
        s = f.read()


      # path_wkhtmltopdf = '/usr/bin/wkhtmltopdf'
      x = !whereis wkhtmltopdf
      path_wkhtmltopdf = x[0].split(": ")[-1]
      config = pdfkit.configuration(wkhtmltopdf=path_wkhtmltopdf)

      try:
        pdfkit.from_string(s, name+'.pdf', configuration=config)
      except:
        pass


      pdf_file_path = name+'.pdf'

      # split pdf into pages
      # Open the PDF file in read-binary mode
      with open(pdf_file_path, 'rb') as pdf_file:
          # Create a PDF reader object
          pdf_reader = PyPDF2.PdfReader(pdf_file)

          # Get the total number of pages in the PDF
          num_pages = len(pdf_reader.pages)

          pages_str = {}
          # Loop through each page
          for page_number in range(num_pages):

              page = pdf_reader.pages[page_number]

              # Extract text from the page (you can do other operations here)
              page_text = page.extract_text()

              # Print or process the page text as needed
              #print(f"Page {page_number + 1}:\n{page_text}\n")
              pages_str[page_number] = page_text
              #pages_str.append(page_text)

      return pages_str
  else:
      print('Failed to fetch the HTML content.')


def report_from_url(url,name):
  """ download webpage as html """
  headers = {"User-Agent": "Mozilla/5.0"}
  response = requests.get(url,headers=headers)

  # Check if the request was successful
  if response.status_code == 200:
      # Parse the HTML content using BeautifulSoup
      soup = BeautifulSoup(response.text, 'html.parser')


      with open(name+'.html','w') as f:
        f.write(response.text)



import os
def pdf_from_url(url, name="output"):
    """Download webpage as a PDF file directly."""
    try:
        counter = 1
        pdf_name = f'{name}.pdf'

        # # Check if the file already exists
        # while os.path.exists(pdf_name):
        #     pdf_name = f'{name}_{counter}.pdf'
        #     counter += 1

        # Convert webpage to PDF
        pdfkit.from_url(url, pdf_name)

        print(f'PDF created successfully: {pdf_name}')

    except Exception as e:
        print(f"Unable to create PDF from webpage. Error: {e}")



def dict_from_pdf(pdf_file_path):
  """ read text data from pdf file directly into dictonary of page_number:content """
  # split pdf into pages
  # Open the PDF file in read-binary mode
  with open(pdf_file_path, 'rb') as pdf_file:
      # Create a PDF reader object
      pdf_reader = PyPDF2.PdfReader(pdf_file)

      # Get the total number of pages in the PDF
      num_pages = len(pdf_reader.pages)

      pages_str = {}
      # Loop through each page
      for page_number in range(num_pages):

          page = pdf_reader.pages[page_number]

          # Extract text from the page (you can do other operations here)
          page_text = page.extract_text()

          # Print or process the page text as needed
          #print(f"Page {page_number + 1}:\n{page_text}\n")
          pages_str[page_number] = page_text
          #pages_str.append(page_text)

  return pages_str


  import fitz  # PyMuPDF

def extract_relevant_page_numbers(pdf_file_path, wanted_funds, unwanted_funds):
    relevant_page_numbers = []
    current_page_relevant = False  # Set the relevant flag to True from the beginning

    if type(wanted_funds) is not list:
      wanted_funds = [wanted_funds]
    if type(unwanted_funds) is not list:
      unwanted_funds = [unwanted_funds]

    # Open the PDF file
    pdf_document = fitz.open(pdf_file_path)

    if len(unwanted_funds) == 0:
      return [i for i in range(pdf_document.page_count)]

    wanted_funds = [fund.lower() for fund in wanted_funds]
    wanted_funds = [re.sub("[^a-z0-9\s]",'',fund) for fund in wanted_funds]
    unwanted_funds = [fund.lower() for fund in unwanted_funds]
    unwanted_funds = [re.sub("[^a-z0-9\s]",'',fund) for fund in unwanted_funds]

    for page_number in range(pdf_document.page_count):
        page = pdf_document.load_page(page_number)
        text = page.get_text("text").lower()
        text = re.sub("[^a-z0-9\s]",'',text)

        # Check if any wanted fund is in the page
        if any(fund in text for fund in wanted_funds):
          current_page_relevant = True

        # Check if no wanted funds are present but unwanted funds are
        if not any(fund in text for fund in wanted_funds) and any(fund in text for fund in unwanted_funds):
          current_page_relevant = False

        # If the page is considered relevant, add its number to the list
        if current_page_relevant:
          relevant_page_numbers.append(page_number)

        # Check if any unwanted fund is in the page
        if any(fund in text for fund in unwanted_funds):
            current_page_relevant = False

    # Close the PDF document
    pdf_document.close()

    return relevant_page_numbers


import os
def pdf_fundpages(url,wanted,unwanted,name="output"):
    """Download webpage as a PDF file directly."""
    try:
        counter = 1
        pdf_name = f'{name}.pdf'

        # Convert webpage to PDF
        pdfkit.from_url(url, pdf_name)

        fund_pages = extract_relevant_page_numbers(pdf_name,wanted,unwanted)

        extract_pages(pdf_name,pdf_name,fund_pages)

        print(f'PDF created successfully: {pdf_name}')


    except Exception as e:
        print(f"Unable to create PDF from webpage. Error: {e}")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [4]:
# use bm25 and get top_k pages
def get_relevant(pages_str,query,top_k):

  # Tokenize the documents (you can use more advanced tokenization methods)
  preprocessed_pages = {index:preprocess(page) for index,page in pages_str.items()}
  tokenized_pages = [doc.split() for doc in preprocessed_pages.values()]

  # Create a BM25 index
  bm25 = BM25Okapi(tokenized_pages)

  # Sample query
  query = query

  # Tokenize the query
  query = preprocess(query)
  tokenized_query = query.split()

  # Get BM25 scores for the query
  scores = bm25.get_scores(tokenized_query)

  # Rank documents by their BM25 scores
  ranked_indices = sorted(range(len(scores)), key=lambda i: -scores[i])
  ranked_documents = {pages_str[i]:scores[i] for i in ranked_indices}

  # Print top k ranked documents
  #top_k = 5
  top_k = top_k
  res = dict(islice(ranked_documents.items(), top_k))

  return dict(zip(ranked_indices[:top_k],sorted(scores,reverse=True)[:top_k]))

In [5]:
import PyPDF2

def extract_pages(input_pdf, output_pdf, page_numbers):
    try:
        # Open the input PDF file
        with open(input_pdf, 'rb') as pdf_file:
            pdf = PyPDF2.PdfReader(pdf_file)
            pdf_writer = PyPDF2.PdfWriter ()

            # Check if the specified page numbers are valid
            valid_page_numbers = [page for page in page_numbers if 0 <= page < len(pdf.pages)]

            valid_page_numbers.sort()

            if not valid_page_numbers:
                print("No valid page numbers specified.")
                return

            # Add the specified pages to the new PDF
            for page in valid_page_numbers:
                pdf_writer.add_page(pdf.pages[page])

            # Write the selected pages to the output PDF
            with open(output_pdf, 'wb') as output_file:
                pdf_writer.write(output_file)

            print(f"Selected pages saved to {output_pdf}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [6]:
import nltk
from nltk import word_tokenize

nltk.download('punkt')

def calculate_recall(reference_text, predicted_text):
    reference_tokens = set(word_tokenize(reference_text.lower()))
    predicted_tokens = set(word_tokenize(predicted_text.lower()))

    true_positives = len(reference_tokens.intersection(predicted_tokens))
    false_negatives = len(reference_tokens - predicted_tokens)

    recall = true_positives / (true_positives + false_negatives)
    return recall

def calculate_precision(reference_text, predicted_text):
    reference_tokens = set(word_tokenize(reference_text.lower()))
    predicted_tokens = set(word_tokenize(predicted_text.lower()))

    true_positives = len(reference_tokens.intersection(predicted_tokens))
    false_positives = len(predicted_tokens - reference_tokens)

    precision = true_positives / (true_positives + false_positives)
    return precision

def select_two_pages(max_ranking_df):
  max_ranking_df = max_ranking_df.sort_values(by='page_num') # expand nodes to get childrens first ->  not done
  filtered_df = pd.DataFrame()
  for index,row in max_ranking_df.iterrows():
    expanded_df = pd.DataFrame({node.text:row.node.page_num for node in get_all_nodes_under(row.node)[1:]}.items(),columns=['node_text','page_num']) # expand each row node to get all its children
    filtered_df = pd.concat([filtered_df,expanded_df],ignore_index=False)

  head_2 = []
  for index,row in filtered_df.iterrows():
      if len(np.unique(head_2)) == 2 and row['page_num'] not in head_2:
        break
      else:
        head_2.append(row['page_num'])

  result_df = max_ranking_df[max_ranking_df['page_num'].isin(head_2)]
  return result_df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
#list of markers and identifiers (structure of text file is known beforehad)
markers =  ["Reporting_Date",
            "Table_Contents",
            "Letter_CEO",
            "Letter_CEO_Title",
            "Letter_SM",
            "Letter_SM_Title",
            "Letter_GEC",
            "Letter_MDFP",
            "GEC",
            "GEC_Title",
            "MDFP",
            "MDFP_Title",
            "MDFP_GEC",
            "Comments_RA"]

identifiers = ["Line_Number", "Series"]

f_date = "Obs_Date"

In [8]:
def getValue(text, splitID, listMarkers):

  dummy = ""

  for w in text:

    if w not in listMarkers and w != splitID:
      dummy += w + " "
    else:
      return dummy[:-1]


  return dummy[:-1]

In [9]:
import re

def buildDict(textFile, listIdentifiers, splitID, listMarkers):

    #open the txt file
    with open(textFile) as f:
        lines = f.read()

    # get Fund_Name
    fund_name = lines.split('\n')[2].split("Fund_Name ")[-1]

    #parse the text file to build a dictionary
    record = {}

    record['fund_name'] = fund_name

    #turn the text file into a list of words
    listWords = lines.split()

    #split the text into topics
    len_text = len(listWords)


    if re.search(splitID, lines):


      for i in range(len_text):

        if listWords[i] == listIdentifiers[0]:
          record[listIdentifiers[0]] = listWords[i+1]

        if listWords[i] == listIdentifiers[1]:
          record[listIdentifiers[1]] = listWords[i+1]


        if listWords[i]  == splitID:
          record[listWords[i + 1]] = {}
          year = listWords[i + 1]

        if listWords[i] in listMarkers:
            record[year][listWords[i]] = getValue(listWords[i+1:], splitID, listMarkers)

    else:

      for i in range(len_text):

        if listWords[i] == listIdentifiers[0]:
          record[listIdentifiers[0]] = listWords[i+1]

        if listWords[i] == listIdentifiers[1]:
          record[listIdentifiers[1]] = listWords[i+1]

        if listWords[i] in listMarkers:
            record[listWords[i]] = getValue(listWords[i+1:], splitID, listMarkers)


    return record


In [10]:
#dates = [i.date for index,i in enumerate(listReports) if index%2==0]
def bm25_ranking_with_preprocessing(query, nodes_texts, top_k):
    corpus = [preprocess_text(text) for text in nodes_texts.values()]
    tokenized_corpus = [doc.split(" ") for doc in corpus]
    bm25 = BM25Okapi(tokenized_corpus)

    query = preprocess_text(query).split(" ")
    scores = bm25.get_scores(query)
    results = tuple((score, node, text,node.tag,node.page_num) for score, (node, text) in zip(scores, nodes_texts.items()))
    results_df = pd.DataFrame(results,columns=['score','node','text','tag','page_num'])
    ranked_df  = results_df[results_df['score']!=0].sort_values(by='score',ascending=False).head(top_k)
    return ranked_df


def wanted_unwanted_funds(edgar_report,wanted_fund):
  funds = [k[1].lower() for k in edgar_report.listFunds] # list all the funds in the report #Fidelity Dividend Growth Fund
  wanted = [x.lower() for x in wanted_fund]
  funds.remove(wanted[0])
  unwanted = funds
  return (wanted,unwanted)

## Defining Functions

In [11]:
# merge two consecutive nodes of type header if they are similary in text "title"
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(str1, str2):
    try:
      vectorizer = CountVectorizer(min_df=1).fit_transform([str1, str2])
      vectors = vectorizer.toarray()
      similarity = cosine_similarity(vectors[0].reshape(1, -1), vectors[1].reshape(1, -1))[0][0]
      return similarity * 100
    except:
      return 0

def merge_nodes(node):
    if node:
        i = 0
        while i < len(node.children) - 1:
            similarity = calculate_similarity(node.children[i].text, node.children[i + 1].text)
            if similarity > 80 and not node.children[i].tag.startswith('p') and not node.children[i].tag.startswith('s'):
                if node.children[i + 1].tag not in ['p', 's']:
                    # Use the text of the first node and include the children of the second node
                    node.children[i].children.extend(node.children[i + 1].children)
                    del node.children[i + 1]
                else:
                    i += 1
            else:
                i += 1
        for child in node.children:
            merge_nodes(child)
    return node  # Return the modified node




from rank_bm25 import BM25Okapi
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import string


# nltk.download('punkt')
# nltk.download('stopwords')

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

def bm25_ranking_with_preprocessing(query, nodes_texts, top_k): # nodes_text is a dict of node:text
    corpus = [preprocess_text(text) for text in nodes_texts.values()]
    tokenized_corpus = [doc.split(" ") for doc in corpus]
    bm25 = BM25Okapi(tokenized_corpus)

    query = preprocess_text(query).split(" ")
    scores = bm25.get_scores(query)
    results = tuple((score, node, text,node.tag, node.page_num) for score, (node, text) in zip(scores, nodes_texts.items()))
    results_df = pd.DataFrame(results,columns=['score','node','text','tag', 'page_num'])
    ranked_df  = results_df[results_df['score']!=0].sort_values(by='score',ascending=False).head(top_k)
    return ranked_df

def get_all_nodes_under(node):
    nodes = [node]
    for child in node.children:
        nodes.extend(get_all_nodes_under(child))
    return nodes

def remove_tags_starting_with_s_or_p(df):
    filtered_df = df[~df['tag'].str.startswith(('s', 'p'))]
    return filtered_df
import pandas as pd

import pandas as pd

def filter_max_children(df):
    max_children_rows = pd.DataFrame(columns=df.columns)
    for score in df['score'].unique():
        temp_df = df[df['score'] == score]
        if len(temp_df) > 1:
            for text in temp_df['text'].unique():
                text_df = temp_df[temp_df['text'] == text]
                if len(text_df) > 1:
                    max_n_children = text_df['n_children'].max()
                    max_children_rows = pd.concat([max_children_rows, text_df[text_df['n_children'] == max_n_children]])
                else:
                    max_children_rows = pd.concat([max_children_rows, text_df])
        else:
            max_children_rows = pd.concat([max_children_rows, temp_df])
    return max_children_rows

In [12]:
def tag_frequency(root_node):
    tag_freq = {}

    # Helper function to traverse the tree and count tag frequencies
    def traverse(node):
        if node.tag in tag_freq:
            tag_freq[node.tag] += 1
        else:
            tag_freq[node.tag] = 1
        for child in node.children:
            traverse(child)

    # Start traversing from the root node
    traverse(root_node)

    return tag_freq
def report_to_graph(url,pdf_name,wanted=[],unwanted=[],edgar_flag=0):
  """ if edgar flag is set, it discards wanted,unwanted parameters and use the links, discarding extraction of fundpages"""
  if edgar_flag:
    pdf_fundpages(url,wanted,unwanted,pdf_name) # list of wanted & unwanted funds, saves fund related pages
  else:
    pdf_from_url(url,pdf_name) #replaced with the remove_tables_from_html_.... line to remove tables from pdf first
  #remove_tables_from_html_link_and_save_as_pdf(url, pdf_name)

  print('done')
  my_file = f"{pdf_name}.pdf"
  doc = fitz.open(my_file)

  output = []

  for page in doc:

      output += page.get_text("blocks")

  previous_block_id = 0 # Set a variable to mark the block id
  blocks = []
  for block in output:

      if block[6] == 0: # We only take the text

            #if previous_block_id != block[5]: # Compare the block number

                #print(">>>>>>>>>>>>>>>>>>>>\n")

            plain_text = cleaning(block[4])
            blocks.append(block)

            #print(plain_text)
  df = pd.DataFrame(blocks,columns=['x0','y0','x1','y1','text','block_no','block_type'])
  df['text'] = df['text'].apply(lambda x:cleaning(x))
  df = df.drop(df[df.block_type == 1 ].index)

  # ===================================

  block_dict = {}

  page_num = 1

  for page in doc: # Iterate all pages in the document

        file_dict = page.get_text('dict') # Get the page dictionary

        block = file_dict['blocks'] # Get the block information

        block_dict[page_num] = block # Store in block dictionary

        page_num += 1 # Increase the page value by 1

# ============================
  import re

  spans = pd.DataFrame(columns=['xmin', 'ymin', 'xmax', 'ymax', 'text', 'tag'])

  rows = []

  for page_num, blocks in block_dict.items():

      for block in blocks:

          if block['type'] == 0:

              for line in block['lines']:

                  for span in line['spans']:



                      xmin, ymin, xmax, ymax = list(span['bbox'])

                      font_size = span['size']

                      text = cleaning(span['text'])

                      span_font = span['font']



                      is_upper = False

                      is_bold = False



                      if "bold" in span_font.lower():

                          is_bold = True




                      if re.sub("[\(\[].*?[\)\]]", "", text).isupper():

                          is_upper = True



                      if text.replace(" ","") !=  "":

                          rows.append((xmin, ymin, xmax, ymax, text, is_upper, is_bold, span_font, font_size,page_num,block['number']))



  span_df = pd.DataFrame(rows, columns=['xmin','ymin','xmax','ymax', 'text', 'is_upper','is_bold','span_font', 'font_size','page_num','block_number'])


# ========================================

  span_scores = []

  span_num_occur = {}

  special = '[(_:/,#%\=@)]'




  for index, span_row in span_df.iterrows():



      score = round(span_row.font_size)

      text = span_row.text



      if not re.search(special, text):



          if span_row.is_bold:

              score +=1




          if span_row.is_upper:

              score +=1



      span_scores.append(score)




  values, counts = np.unique(span_scores, return_counts=True)

  #==================================


  style_dict = {}

  for value, count in zip(values, counts):

      style_dict[value] = count

  sorted(style_dict.items(), key=lambda x: x[1])


  # =============================

  p_size = max(style_dict, key=style_dict.get)
  #p_size = span_df[span_df['text'].str.contains('dear', case=False)].font_size.values[0]+1




  idx = 0

  tag = {}




  for size in sorted(values, reverse = True):

      idx += 1

      if size == p_size:

          idx = 0

          tag[size] = 'p'

      if size > p_size:

          tag[size] = 'h{0}'.format(idx)

      if size < p_size:

          tag[size] = 's{0}'.format(idx)


  # =================================

  span_tags = [tag[score] for score in span_scores]

  span_df['tag'] = span_tags



  def split_dataframe_page_numbers(df):
    # takes a dataframe as input and group items by page_num then block_number to be used later (in merge_procedure) to merge text items in the same block
    # this function has drawback, that it fails in the case of small page size, since multiple small pdf pages are extracted as one page, so it combines
    # text items in the same page and block, then merge_prcedure merge them, oversighting the fact that they come from different pdf pages
    split_dfs = []
    for (block_number, page_num), group in df.groupby(['page_num','block_number']): # modified, i changed it to put page_num first
        split_dfs.append(group)
    return split_dfs

  def merge_and_append(merge, new):
      if len(merge) > 0:
        combined_merge = combine_text_blocks(merge)
        result = pd.concat([combined_merge, new], ignore_index=True)
      else:
        result = new
      return result

  def combine_text_blocks(df):
      tag_order = {'h0': 0, 'h1': 1, 'h2': 2, 'h3': 3, 'h4': 4, 'h5': 5,'h6':6,'h7':7,'h8':8,'h9':9,'h10':10,'h11':11, 'h12': 12,'h13':13, 'p': 14,'s':15, 's1': 16,
                's2':17,'s3':18, 's4': 19,'s5':20, 's6': 21, 's7': 22}
      combined_df = df.groupby(['block_number', 'page_num']).apply(lambda x: pd.Series({ # doesn't matter which is first as long as we pass rows for a same page and bock
          'xmin': x['xmin'].iloc[0],
          'xmax': x['xmax'].iloc[0],
          'ymin': x['ymin'].iloc[0],
          'ymax': x['ymax'].iloc[0],
          'tag': x['tag'][x['tag'].map(tag_order).idxmin()], # modified 28/2/2024, we set the tag of the merged rows to be the max tag (min bec. order reversed)
          'font_size': x['font_size'].iloc[0],
          'is_bold': x['is_bold'].iloc[0],
          'is_upper': x['is_upper'].iloc[0],
          'span_font': x['span_font'].iloc[0],
          'text': combine_text(x['text'], x['tag'], tag_order)
      })).reset_index()
      return combined_df.sort_values(by=['page_num', 'block_number']).reset_index(drop=True)

  def combine_text(text_series, tag_series, tag_order):
      combined_text = text_series.iloc[0]  # Initialize with the first text
      reference_tag = tag_series.iloc[0]


      for i in range(1, len(text_series)):
          current_tag = tag_series.iloc[i]
          if current_tag == reference_tag:
              combined_text += ' ' + text_series.iloc[i]
          else:
              if tag_order[current_tag] > tag_order[reference_tag]:
                  combined_text += '' + text_series.iloc[i]
              else:
                  combined_text += ' ' + text_series.iloc[i]
          #reference_tag = current_tag
      return combined_text


  def merge_procedure(df):
    """ takes a dataframe, combine headers until hit p,s or any long sentence """
    blocks_pages = split_dataframe_page_numbers(df)
    df_combined = pd.DataFrame()
    for block_page in blocks_pages:
      combine = []
      skip_combine = 0
      new = []
      for index,row in block_page.iterrows():
        if len(row.text.split(" "))<4 and skip_combine == 0:
          combine.append(row)
        else:
          new.append(row)
          skip_combine=1

      current_df = merge_and_append(pd.DataFrame(combine),pd.DataFrame(new))
      df_combined = pd.concat([df_combined, current_df], ignore_index=True)
    df_combined = df_combined.sort_values(by=['page_num', 'block_number']).reset_index(drop=True)
    return df_combined

  # span_df = combine_text_blocks(span_df)
  span_df = merge_procedure(span_df)
# ==========================================

  def merge_consecutive_nodes_with_same_tag(node):
    """merge two consecutive nodes with the same name and tag"""
    if node:
        i = 0
        while i < len(node.children) - 1:
            if node.children[i].tag == node.children[i + 1].tag and (node.children[i].text == node.children[i + 1].text or node.children[i].text == node.children[i + 1].text):
                # node.children[i].text += node.children[i + 1].text # this will duplicate the text/title
                node.children[i].children.extend(node.children[i + 1].children)
                del node.children[i + 1]
            else:
                i += 1
        for child in node.children:
            merge_consecutive_nodes_with_same_tag(child)
    return node

  # working as it returns linkedlist but it may doesn't add all nodes in the linkedlist # worked !
  class Node:
      def __init__(self, text, tag, page_num):
          self.text = text
          self.tag = tag
          self.page_num = page_num
          self.parent = None
          self.children = []

  # Sample data
  # data = {
  #     'text': ['root','ahmed', 'goes', 'to', 'by', 'foot', 'but', 'he', 'is', 'late','for','school','right'],
  #     'tag': ['h0','h1', 'h2', 'h2', 'h3', 'p', 'h1', 's', 'h3', 'h2','p','h4','h5']
  # }
  x=span_df['text'].tolist()
  x.insert(0,'root')
  y=span_df['tag'].tolist()
  y.insert(0,'h0') # insert tag h0 at index '0' for the root node to be the parent of all the nodes in the tree
  z=span_df['page_num'].tolist()
  z.insert(0,1) # at position 0 which is the root, assign page_num of value page_num=1


  data ={

      "text": x,
      "tag" : y,
      'page_num' : z
  }

  # Define the tag order dictionary
  tag_order = {'h0': 0, 'h1': 1, 'h2': 2, 'h3': 3, 'h4': 4, 'h5': 5,'h6':6,'h7':7,'h8':8,'h9':9,'h10':10,'h11':11, 'h12': 12,'h13':13, 'p': 14,'s':15, 's1': 16,
              's2':17,'s3':18, 's4': 19,'s5':20, 's6': 21, 's7': 22}

  # Create nodes for each data point
  nodes = [Node(text, tag, page_num) for (text, tag,page_num) in zip(data['text'], data['tag'], data['page_num'])]

  # Build the linked list
  for i in range(1, len(nodes)):
      current_node = nodes[i]
      parent = None
      for j in range(i - 1, -1, -1):
          if tag_order[nodes[j].tag] < tag_order[current_node.tag]:
              parent = nodes[j]
              break
      if parent:
          current_node.parent = parent
          parent.children.append(current_node)
      else:
          for j in range(i - 1, -1, -1):
              if tag_order[nodes[j].tag] > tag_order[current_node.tag]:
                  current_node.parent = nodes[j].parent
                  nodes[j].parent = current_node
                  current_node.children.append(nodes[j])
                  break

  # Find the root node
  root = nodes[0]
  while root.parent:
      root = root.parent


  root = merge_consecutive_nodes_with_same_tag(root)
  frequency_of_tags = tag_frequency(root)

  # Call the function on the root node to start the merging process
  #modified_root = merge_nodes(root)

  #return span_df,frequency_of_tags,modified_root
  return span_df,frequency_of_tags,root

In [13]:
def get_all_nodes_except_p_s(root):
    result = []

    def dfs(node):
        if not node.text.startswith('p') and not node.text.startswith('s'):
            result.append(node)
        for child in node.children:
            dfs(child)

    dfs(root)
    return result

In [ ]:
import pandas as pd

def increase_page_num_on_decreasing_block_num(df):
    # Group by 'page_num'
    grouped_df = df.groupby('page_num')

    # Initialize an empty DataFrame to store the modified rows
    modified_rows = pd.DataFrame()

    # Iterate through each group
    for page_num, group in grouped_df:
        # Check for decreasing 'block_num'
        decreasing_block_num = group['block_num'].diff() < 0

        # If any row has decreasing 'block_num', increase 'page_num' for the rows underneath
        if decreasing_block_num.any():
            # Identify the indices where 'block_num' is decreasing
            indices_to_increase = group[decreasing_block_num].index

            # Increase 'page_num' for the rows underneath
            for idx in indices_to_increase:
              df.loc[idx:, 'page_num'] += 1

            # Append the modified rows to the DataFrame
            modified_rows = modified_rows.append(df.loc[indices_to_increase])

    return modified_rows

# Example usage
data = {'block_num': [1, 2, 1, 1, 2, 1, 2, 3, 1, 1],
        'page_num': [1, 1, 1, 1, 1, 2, 2, 2, 2, 2]}

df = pd.DataFrame(data)
print("Original DataFrame:")
print(df)

# modified_rows = increase_page_num_on_decreasing_block_num(df)
# print("\nModified DataFrame:")
# print(df)
# print("\nModified Rows:")
# print(modified_rows)


## Automate multiple annotation files

In [14]:
#list of markers and identifiers (structure of text file is known beforehad)
markers =  ["Reporting_Date",
            "Table_Contents",
            "Letter_CEO",
            "Letter_CEO_Title",
            "Letter_SM",
            "Letter_SM_Title",
            "Letter_GEC",
            "Letter_MDFP",
            "GEC",
            "GEC_Title",
            "MDFP",
            "MDFP_Title",
            "MDFP_GEC",
            "Comments_RA"]

identifiers = ["Line_Number", "Series"]

f_date = "Reporting_Date"

In [15]:
def getValue(text, splitID, listMarkers):

  dummy = ""

  for w in text:

    if w not in listMarkers and w != splitID:
      dummy += w + " "
    else:
      return dummy[:-1]


  return dummy[:-1]

In [16]:
import re

def buildDict(textFile, listIdentifiers, splitID, listMarkers):

    #open the txt file
    with open(textFile) as f:
        lines = f.read()

    #parse the text file to build a dictionary
    record = {}



    #turn the text file into a list of words
    listWords = lines.split()

    #split the text into topics
    len_text = len(listWords)

    flag_series = 0
    if re.search(splitID, lines):


      for i in range(len_text):

        if listWords[i] == listIdentifiers[0]:
          record[listIdentifiers[0]] = listWords[i+1]

        if listWords[i] == listIdentifiers[1] and flag_series == 0:
          flag_series=1
          record[listIdentifiers[1]] = listWords[i+1]


        if listWords[i]  == splitID:
          record[listWords[i + 1]] = {}
          year = listWords[i + 1]

        if listWords[i] in listMarkers:
            record[year][listWords[i]] = getValue(listWords[i+1:], splitID, listMarkers)

    else:

      for i in range(len_text):

        if listWords[i] == listIdentifiers[0]:
          record[listIdentifiers[0]] = listWords[i+1]

        if listWords[i] == listIdentifiers[1] and flag_series == 0:
          flag_series = 1
          record[listIdentifiers[1]] = listWords[i+1]

        if listWords[i] in listMarkers:
            record[listWords[i]] = getValue(listWords[i+1:], splitID, listMarkers)


    # get Fund_Name
    return record


In [ ]:
# loop through annotation files
from google.colab import files
from glob import glob
import time


textual_dict = {}         # this is for saving textual elements df of different files, it uses text_dict_dates dictionary
one_sample_setting = 1
MDFP_Title = 1
if one_sample_setting:
  print("One_Sample_Setting")
if MDFP_Title:
  print("MDFP titles included only")

total_df = pd.DataFrame(columns = ["date", "truth",'extracted','tree'])

extracted_files = glob('*.txt')
for index,file in enumerate(extracted_files):
  # ==============
  print(index+1,' ======== ',file)
  records = buildDict(file,identifiers,f_date,markers)


  # ==============
  one_year_fund = 0
  multi_year_fund = 0
  try: # to handle any missing field or different strucutre
    ann_date_mdfp = {}
    for index,(k,v) in enumerate(records.items()):
      if index > 2:
        if 'MDFP' in v:
          ann_date_mdfp[k] = v['MDFP']
          multi_year_fund+=1
        elif 'MDFP' in records:
          ann_date_mdfp[k] = records['MDFP']
          one_year_fund+=1
  except Exception as ex:
    print("Missing file or Different structure txt file")
    continue

  ann = pd.DataFrame(ann_date_mdfp.items(),columns = ["date", "truth"])
  ann['extracted'] = ''

  ann = ann[ann.truth!='NA']
  ann['year'] = ann.date.apply(lambda x:x.split('-')[0])
  ann = ann.groupby('year').first().reset_index()
  ann['file'] = file

  try:
    sn = records['Series']
    if MDFP_Title:   # if specified, then select only years where MDFP section has title
      x = pd.DataFrame(records).transpose().drop(['Series','Line_Number'],axis=0)
      x = x[x.MDFP_Title!='NA']
      records = x.transpose().to_dict()
    # sn = records['Series']
    listReports = searchEdgarNCSR(sn, 100)
    listReports = [i for i in listReports if i.filing=='N-CSR']

    wanted_fund = [listReports[0].name]
    dates = []
    wanted = []
    unwanted = []
    links = []
    for i in range(len(listReports)):
      try:
        w,u = wanted_unwanted_funds(listReports[i],wanted_fund)
        dates.append(listReports[i].reportingDate)
        links.append(listReports[i].link)
        wanted.append(w)
        unwanted.append(u)
      except Exception as e: # fund doesn't exist in this year
        print(e)
        print(f"Can't find wanted fund in {listReports[i].reportingDate}")


    #edgar_df = pd.DataFrame(np.column_stack([dates,links,wanted,unwanted]),columns=['date','link','wanted','unwanted'])
    edgar_df = pd.DataFrame({'date':dates,'link':links,'wanted':wanted,'unwanted':unwanted, 'tree':'NA'})
    edgar_df.date = pd.to_datetime(edgar_df.date).apply(lambda x: x.strftime('%Y-%m-%d')) # make it standard date to facilitate the merge
    ann.date = pd.to_datetime(ann.date).apply(lambda x: x.strftime('%Y-%m-%d'))           # make it standard date to facilitate the merge
    edgar_df = edgar_df.merge(ann,on='date',how='inner') # inner join "merge" on date column
    text_dict_dates = {}      # this is for saving textual elements dataframe of specific file, and initialize every new file


    if one_sample_setting:                            # if specified, then use the most recent year only
      edgar_df = edgar_df[edgar_df.year == ann.year.values.max()] # to set only the latest year according to ann last 'annotation' year



    results = []
    for index,row in edgar_df.iterrows():
      try:
        # df,style,root = report_to_graph(url,f"{url.replace('/','$')}_{index}",wanted,unwanted)
        df,style,root = report_to_graph(row.link,f'{row.wanted[0]}_{row.date.split("-")[0]}',row.wanted,row.unwanted,1)
        # df, style, root = report_to_graph(url,f"{url.replace('/','$')}_{index}") # report_to_graph function is defined bellow
        results.append([row.date,df,style,root])
        text_dict_dates[row.date] = df # add textual dataframe
      except Exception as e:
        print(f" ============= Error encountered ======== {e}")


    # add automatic extraction to the dataframe "edgar_df" containing date, true MDFP for easy comparison
    for i,report in enumerate(results):

      all_nodes = get_all_nodes_except_p_s(report[2+1]) # bec. we added date at first index
      all_nodes  = {node:node.text for node in all_nodes}

      query = "fund performance discussion commentary"
      #query = "fund portfolio commentary management performance discussion"
      top_k = 100
      ranking = bm25_ranking_with_preprocessing(query, all_nodes, top_k)
      ranking['n_children']=ranking.node.apply(lambda x:len(x.children))
      filtered_ranking = ranking[ranking['n_children']>0] # filter out matches with no children
      # filtered_ranking = filter_max_children(filtered_ranking)

      filtered_ranking = remove_tags_starting_with_s_or_p(filtered_ranking)
      max_rank = filtered_ranking.score.max()
      max_ranking_df = filtered_ranking[filtered_ranking.score == max_rank]
      max_ranking_df = select_two_pages(max_ranking_df)
      text = ''
      for index,row in max_ranking_df.iterrows():
          text += '\n'.join([node.text for node in get_all_nodes_under(row.node)[1:]])
          text+='\n\n'

      print(links[i])
      print(edgar_df.date.values[i])
      edgar_df['extracted'][edgar_df.date == report[0]] = text
      edgar_df['tree'][edgar_df.date == report[0]] = report[2+1] # add the root node in the dataframe



      print("==============================")
    textual_dict[file] =  text_dict_dates
    total_df = pd.concat([total_df,edgar_df])

    print(f" =============== Finished file : {file} =================")
  except Exception as e:
    # print(e)
    print("couldn't retrive from edgar database")

In [ ]:
# filter the empty extracted
empty_df = total_df[total_df.extracted==''] # save empty extracted in new dataframe
total_df = total_df[total_df.extracted!=''] # remove empty extracted
total_df.shape

(0, 4)

In [ ]:
total_df.shape

(85, 9)

In [ ]:
import pandas as pd
# total_df.to_csv('data.csv')
total_df = pd.read_csv("data.csv")
total_df.shape

(162, 10)

## Experiment 1
Fix the year and run on all the reports

In [ ]:
#list of markers and identifiers (structure of text file is known beforehad)
markers =  ["Reporting_Date",
            "Table_Contents",
            "Letter_CEO",
            "Letter_CEO_Title",
            "Letter_SM",
            "Letter_SM_Title",
            "Letter_GEC",
            "Letter_MDFP",
            "GEC",
            "GEC_Title",
            "MDFP",
            "MDFP_Title",
            "MDFP_GEC",
            "Comments_RA"]

identifiers = ["Line_Number", "Series"]

f_date = "Reporting_Date"

In [ ]:
def getValue(text, splitID, listMarkers):

  dummy = ""

  for w in text:

    if w not in listMarkers and w != splitID:
      dummy += w + " "
    else:
      return dummy[:-1]


  return dummy[:-1]

In [ ]:
import re

def buildDict(textFile, listIdentifiers, splitID, listMarkers):

    #open the txt file
    with open(textFile) as f:
        lines = f.read()

    #parse the text file to build a dictionary
    record = {}



    #turn the text file into a list of words
    listWords = lines.split()

    #split the text into topics
    len_text = len(listWords)

    flag_series = 0
    if re.search(splitID, lines):


      for i in range(len_text):

        if listWords[i] == listIdentifiers[0]:
          record[listIdentifiers[0]] = listWords[i+1]

        if listWords[i] == listIdentifiers[1] and flag_series == 0:
          flag_series=1
          record[listIdentifiers[1]] = listWords[i+1]


        if listWords[i]  == splitID:
          record[listWords[i + 1]] = {}
          year = listWords[i + 1]

        if listWords[i] in listMarkers:
            record[year][listWords[i]] = getValue(listWords[i+1:], splitID, listMarkers)

    else:

      for i in range(len_text):

        if listWords[i] == listIdentifiers[0]:
          record[listIdentifiers[0]] = listWords[i+1]

        if listWords[i] == listIdentifiers[1] and flag_series == 0:
          flag_series = 1
          record[listIdentifiers[1]] = listWords[i+1]

        if listWords[i] in listMarkers:
            record[listWords[i]] = getValue(listWords[i+1:], splitID, listMarkers)


    # get Fund_Name
    return record


In [ ]:
# loop through annotation files
from google.colab import files
from glob import glob
import time


textual_dict = {}         # this is for saving textual elements df of different files, it uses text_dict_dates dictionary
one_sample_setting = 0
MDFP_Title = 0
year = '2018'
missing_funds = []         # list to store funds not found in a specific year or the name is different. [(missing_funds, year)]
structure_failure = []     # list to store files that w fail to automatically read bec. of old structure(peu-etre)
db_issue = []              # list to store files that includes funds we can't retrieve from edgar DB
if one_sample_setting:
  print("One_Sample_Setting")
if MDFP_Title:
  print("MDFP titles included only")

total_df = pd.DataFrame(columns = ["date", "truth",'extracted','tree'])

extracted_files = glob('*.txt')
for index,file in enumerate(extracted_files):
  # ==============
  print(index+1,' ======== ',file)
  records = buildDict(file,identifiers,f_date,markers)


  # ==============
  one_year_fund = 0
  multi_year_fund = 0
  try: # to handle any missing field or different strucutre
    ann_date_mdfp = {}
    for index,(k,v) in enumerate(records.items()):
      if index > 2:
        if 'MDFP' in v:
          ann_date_mdfp[k] = v['MDFP']
          multi_year_fund+=1
        elif 'MDFP' in records:
          ann_date_mdfp[k] = records['MDFP']
          one_year_fund+=1
  except Exception as ex:
    print("Missing file or Different structure txt file")
    continue

  ann = pd.DataFrame(ann_date_mdfp.items(),columns = ["date", "truth"])
  ann['extracted'] = ''

  ann = ann[ann.truth!='NA']
  ann['year'] = ann.date.apply(lambda x:x.split('-')[0])
  ann = ann.groupby('year').first().reset_index()
  ann['file'] = file

  try:
    sn = records['Series']
    if MDFP_Title:   # if specified, then select only years where MDFP section has title
      x = pd.DataFrame(records).transpose().drop(['Series','Line_Number'],axis=0)
      x = x[x.MDFP_Title!='NA']
      records = x.transpose().to_dict()
    # sn = records['Series']
    listReports = searchEdgarNCSR(sn, 100)
    listReports = [i for i in listReports if i.filing=='N-CSR']

    wanted_fund = [listReports[0].name]
    dates = []
    wanted = []
    unwanted = []
    links = []
    for i in range(len(listReports)):
      try:
        w,u = wanted_unwanted_funds(listReports[i],wanted_fund)
        dates.append(listReports[i].reportingDate)
        links.append(listReports[i].link)
        wanted.append(w)
        unwanted.append(u)
      except Exception as e: # fund doesn't exist in this year
        print(e)
        print(f"Can't find wanted fund in {listReports[i].reportingDate}")
        missing_funds.append((w[0],listReports[i].reportingDate))


    #edgar_df = pd.DataFrame(np.column_stack([dates,links,wanted,unwanted]),columns=['date','link','wanted','unwanted'])
    edgar_df = pd.DataFrame({'date':dates,'link':links,'wanted':wanted,'unwanted':unwanted, 'tree':'NA'})
    edgar_df.date = pd.to_datetime(edgar_df.date).apply(lambda x: x.strftime('%Y-%m-%d')) # make it standard date to facilitate the merge
    ann.date = pd.to_datetime(ann.date).apply(lambda x: x.strftime('%Y-%m-%d'))           # make it standard date to facilitate the merge
    edgar_df = edgar_df.merge(ann,on='date',how='inner') # inner join "merge" on date column
    # ============= experiment 1: choose a year and fix ===============
    edgar_df = edgar_df[edgar_df.date.str.split('-').apply(lambda x:x[0]) == year]
    if len(edgar_df)>0:
      print(f"year {year} found ...")
    else:
      print(f"year {year} not found ..!")
    #==================================================================
    text_dict_dates = {}      # this is for saving textual elements dataframe of specific file, and initialize every new file


    if one_sample_setting:                            # if specified, then use the most recent year only
      edgar_df = edgar_df[edgar_df.year == ann.year.values.max()] # to set only the latest year according to ann last 'annotation' year



    results = []
    for index,row in edgar_df.iterrows():
      try:
        # df,style,root = report_to_graph(url,f"{url.replace('/','$')}_{index}",wanted,unwanted)
        df,style,root = report_to_graph(row.link,f'{row.wanted[0]}_{row.date.split("-")[0]}',row.wanted,row.unwanted,1)
        # df, style, root = report_to_graph(url,f"{url.replace('/','$')}_{index}") # report_to_graph function is defined bellow
        results.append([row.date,df,style,root])
        text_dict_dates[row.date] = df # add textual dataframe
      except Exception as e:
        print(f" ============= Error encountered ======== {e}")


    # add automatic extraction to the dataframe "edgar_df" containing date, true MDFP for easy comparison
    for i,report in enumerate(results):

      all_nodes = get_all_nodes_except_p_s(report[2+1]) # bec. we added date at first index
      all_nodes  = {node:node.text for node in all_nodes}

      query = "fund performance discussion commentary"
      #query = "fund portfolio commentary management performance discussion"
      top_k = 100
      ranking = bm25_ranking_with_preprocessing(query, all_nodes, top_k)
      ranking['n_children']=ranking.node.apply(lambda x:len(x.children))
      filtered_ranking = ranking[ranking['n_children']>0] # filter out matches with no children
      # filtered_ranking = filter_max_children(filtered_ranking)

      filtered_ranking = remove_tags_starting_with_s_or_p(filtered_ranking)
      max_rank = filtered_ranking.score.max()
      max_ranking_df = filtered_ranking[filtered_ranking.score == max_rank]
      max_ranking_df = select_two_pages(max_ranking_df)
      text = ''
      for index,row in max_ranking_df.iterrows():
          text += '\n'.join([node.text for node in get_all_nodes_under(row.node)[1:]])
          text+='\n\n'

      print(links[i])
      print(edgar_df.date.values[i])
      edgar_df['extracted'][edgar_df.date == report[0]] = text
      edgar_df['tree'][edgar_df.date == report[0]] = report[2+1] # add the root node in the dataframe



      print("==============================")
    textual_dict[file] =  text_dict_dates
    total_df = pd.concat([total_df,edgar_df])

    print(f" =============== Finished file : {file} =================")
  except Exception as e:
    # print(e)
    db_issue.append(file)
    print("couldn't retrive from edgar database")

1  ========  P644.txt
couldn't retrive from edgar database
2  ========  P1099.txt
couldn't retrive from edgar database
3  ========  P624.txt
list.remove(x): x not in list
Can't find wanted fund in 2021-09-30
list.remove(x): x not in list
Can't find wanted fund in 2020-09-30
list.remove(x): x not in list
Can't find wanted fund in 2019-09-30
list.remove(x): x not in list
Can't find wanted fund in 2018-09-30
list.remove(x): x not in list
Can't find wanted fund in 2017-09-30
list.remove(x): x not in list
Can't find wanted fund in 2016-09-30
list.remove(x): x not in list
Can't find wanted fund in 2015-09-30
list.remove(x): x not in list
Can't find wanted fund in 2014-09-30
list.remove(x): x not in list
Can't find wanted fund in 2013-09-30
list.remove(x): x not in list
Can't find wanted fund in 2012-09-30
list.remove(x): x not in list
Can't find wanted fund in 2011-09-30
list.remove(x): x not in list
Can't find wanted fund in 2010-09-30
list.remove(x): x not in list
Can't find wanted fund in

<ipython-input-17-397a134dec81>:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ranking['n_children']=ranking.node.apply(lambda x:len(x.children))


https://www.sec.gov/Archives/edgar/data/745463/000119312524050062/d627480dncsr.htm
2018-12-31
 =============== Finished file : P267.txt =================
81  ========  P714.txt
year 2018 found ...
Selected pages saved to emerald insights fund_2018.pdf
PDF created successfully: emerald insights fund_2018.pdf
done
https://www.sec.gov/Archives/edgar/data/915802/000139834423012831/fp0084043-1_ncsr.htm
2018-04-30
 =============== Finished file : P714.txt =================
82  ========  P121.txt
year 2018 not found ..!
 =============== Finished file : P121.txt =================
83  ========  P162.txt
couldn't retrive from edgar database
84  ========  P85.txt
year 2018 not found ..!
 =============== Finished file : P85.txt =================
85  ========  P637.txt
list.remove(x): x not in list
Can't find wanted fund in 2022-07-31
list.remove(x): x not in list
Can't find wanted fund in 2021-07-31
couldn't retrive from edgar database
86  ========  P615.txt
list.remove(x): x not in list
Can't fin

In [ ]:
total_df.shape

(157, 9)

In [ ]:
total_df.to_csv('total_df.csv',index=False)

In [ ]:
len(db_issue)

118

In [ ]:
len(missing_funds)

1767

In [ ]:
len(structure_failure)

0

## Evaluation

In [ ]:
#!pip install rouge
import nltk,re
from nltk.translate.bleu_score import sentence_bleu,SmoothingFunction
from rouge import Rouge
import pandas as pd
import sys
sys.setrecursionlimit(5000)



def calculate_scores(df):
    bleu_scores = []
    rouge_scores = []
    recall_scores = []
    precision_scores = []
    included_scores = []

    rouge = Rouge()

    for index, row in df.iterrows():
        # ground = row['extracted']
        # ground = re.sub("[^a-z0-9]\s",' ',' '.join(ground.split()).lower())
        # truth = row['truth']
        # truth = re.sub("[^a-z0-9]\s",' ',' '.join(truth.split()).lower())

        # Calculate BLEU score
        # bleu = sentence_bleu([truth.split()], ground.split())


        try:
          extracted = row['extracted']
          extracted = ' '.join(re.sub("[^a-z0-9]",' ',' '.join(extracted.split()).lower()).split())
          truth = row['truth']
          truth = ' '.join(re.sub("[^a-z0-9]",' ',' '.join(truth.split()).lower()).split())



          # Define a smoothing function
          smooth = SmoothingFunction().method1
          # Calculate BLEU score with smoothing
          bleu = sentence_bleu([truth.split()], extracted.split(), smoothing_function=smooth)
          # Calculate ROUGE score
          rouge_score = rouge.get_scores(extracted, truth)


        except:
          extracted = row['extracted']
          extracted = re.sub("[^a-z0-9]\s",' ',' '.join(extracted.split()).lower())
          truth = row['truth']
          truth = re.sub("[^a-z0-9]\s",' ',' '.join(truth.split()).lower())

          bleu = sentence_bleu([truth.split()], extracted.split(), smoothing_function=smooth)
          # Calculate ROUGE score
          rouge_score = rouge.get_scores(extracted, truth)

        # append outside the loop
        rouge_scores.append(rouge_score[0]['rouge-l']['f'])
        bleu_scores.append(bleu)

        x = ' '.join(re.sub("[^a-z0-9]",' ',' '.join(row['extracted'].split()).lower()).split()) # x is the automatic extracted
        y = ' '.join(re.sub("[^a-z0-9]",' ',' '.join(row['truth'].split()).lower()).split()) # y is the annotation
        included_scores.append(y in x)
        # calculate precision and recall
        recall_scores.append(calculate_recall(x,y))
        precision_scores.append(calculate_precision(x,y))




    # Create a new dataframe with BLEU and ROUGE scores
    result_df = df.copy().drop(['unwanted'],axis=1)
    result_df['bleu'] = bleu_scores
    result_df['rouge'] = rouge_scores
    result_df['recall'] = recall_scores
    result_df['precision'] = precision_scores

    result_df['included'] = included_scores # check if the reference as whole is part of the extracted (this is when extracted is covering the reference but )

    return result_df

# Calculate scores
# result_df = calculate_scores(ann).sort_values(by=['rouge','bleu'],ascending=False)
# result_df = calculate_scores(total_df).sort_values(by=['rouge','bleu'],ascending=False)
result_df = calculate_scores(df1).sort_values(by=['rouge','bleu'],ascending=False)
result_df

In [ ]:
print(f'mean rouge {result_df.rouge.mean()}')
print(f'std rouge {result_df.rouge.std()}')
print(f'mean precision {result_df.precision.mean()}')
print(f'std precision {result_df.precision.std()}')
print(f'mean recall {result_df.recall.mean()}')
print(f'std recall {result_df.recall.std()}')

mean rouge 0.4126860729651073
std rouge 0.36778866791703074
mean precision 0.5449696514346191
std precision 0.4245477561122437
mean recall 0.5738016585155327
std recall 0.3390437491493076


In [ ]:
result_df.included.mean()
print(f'std included {result_df.included.std()}')

std included 0.4288588868025356


In [ ]:
import seaborn as sns
sns.distplot(df.rouge)

In [ ]:
import seaborn as sns
sns.distplot(df.precision)

In [ ]:
import seaborn as sns
sns.histplot(df.included)

In [ ]:
ax = sns.distplot(df['included'], kde=False, bins=8,color='black')

for p in ax.patches:
    ax.annotate(f'{p.get_height():.0f}\n',
                (p.get_x() + p.get_width() / 2, p.get_height()), ha='center', va='center', color='crimson')
plt.show()